# Using the H$\delta_A$ and $D_n 4000\,Å$ break indices to quantify the "burstiness" of the SFH
---
Kauffmann et al. (2003) found that stellar continua with "bursty" and continunous SFHs can be distinguished by plotting the H$\delta_A$ index against the $D_n 4000\,Å$ break strength. 

In this notebook, we will 
* Define functions for computing both stellar indices;
* use the Gonzalez-Delgado stellar templates to create "bursty" and continuous SFHs;
* Measure the H$\delta_A$ and $D_n 4000\,Å$ break strengths of these spectra, and recreate their Figs. 2 and 3, to check that the same relationships exist with our templates;
* Investigate the effects of an AGN continuum on these figures;
* Compute these indices for Phil's spectra and see if they lie in the expected regions of the H$\delta_A$ vs. $D_n 4000\,Å$ break strength diagram;
* Using ppxf, remove emission lines from noisy mock spectra & check that the indices can accurately be recovered.

In [2]:
%matplotlib widget

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [66]:
import numpy as np
from scipy import constants
from tqdm.notebook import tqdm

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates, get_bin_edges_and_widths
from ppxftests.mockspec import create_mock_spectrum
from ppxftests.sfhutils import load_sfh, convert_mass_weights_to_light_weights
from ppxftests.sfhutils import compute_mw_age, compute_lw_age, compute_sfr_thresh_age, compute_sb_zero_age, compute_mass
from ppxftests.ppxf_plot import plot_sfh_mass_weighted, plot_sfh_light_weighted


import matplotlib.pyplot as plt 
plt.ion()
plt.close("all")

In [45]:
###################################################
# For testing: create a spectrum
###################################################
isochrones = "Padova"
SNR = 200
z = 0.01

sfh_mw_input, sfh_lw_input, sfr_avg_input, sigma_star_kms = load_sfh(gal=99, plotit=True)

spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=False,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)


/home/u5708159/python/Modules/ppxftests/ppxf_plot.py:32: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 3.5))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/ppxftests/ppxf_plot.py:72: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 3.5))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/ppxftests/ppxf_plot.py:110: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 3.5))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/ppxftests/mockspec.py:179: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/ppxftests/mockspec.py:399: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
###################################################
# Function for computing the HdeltaA index 
# as per Worthey & Ottaviani (1997)
###################################################
def compute_Hdelta(spec, lambda_vals_A, z):
    # De-redshift the input spectrum
    lambda_vals_rest_A = lambda_vals_A / (1 + z) 

    # Define the blue & red passbands, plus the midpoint
    lambda_start_b_A = 4041.60
    lambda_stop_b_A = 4079.75
    lambda_mid_b_A = 0.5 * (lambda_start_b_A + lambda_stop_b_A)
    lambda_start_r_A = 4128.50 
    lambda_stop_r_A = 4161.00
    lambda_mid_r_A = 0.5 * (lambda_start_r_A + lambda_stop_r_A)
    
    # Central passband
    lambda_start_c_A = 4083.50 
    lambda_stop_c_A = 4122.25
    lambda_mid_c_A = 0.5 * (lambda_start_c_A + lambda_stop_c_A)

    # Determine the corresponding indices in the input spectrum
    lambda_start_b_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_start_b_A))
    lambda_stop_b_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_stop_b_A)) 
    lambda_start_r_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_start_r_A))
    lambda_stop_r_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_stop_r_A)) 
    lambda_mid_b_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_mid_b_A)) 
    lambda_mid_r_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_mid_r_A)) 
    lambda_start_c_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_start_c_A))
    lambda_stop_c_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_stop_c_A)) 

    # Compute the mean flux in the blue & red passbands 
    f_mean_b = np.nanmean(spec[lambda_start_b_idx:lambda_stop_b_idx])
    f_mean_r = np.nanmean(spec[lambda_start_r_idx:lambda_stop_r_idx])

    # Define the continuum within the central passband to be halfway between the mean fluxes in the red & blue passbands 
    f_mean_c = 0.5 * (f_mean_b + f_mean_r)

    # Integrate the spectrum from lambda_start_c_A to lambda_stop_c_A to compute the EW
    dlambda_rest_A = lambda_vals_rest_A[1] - lambda_vals_rest_A[0]
    A_spec = dlambda_rest_A * np.nansum(spec[lambda_start_c_idx:lambda_stop_c_idx])

    # Integrate the corresponding area under the pseudocontinuum
    m = (f_mean_b - f_mean_r) / (lambda_mid_b_A - lambda_mid_r_A)
    b = f_mean_b - m * lambda_mid_b_A
    y = lambda l: m * l + b
    A_trapezoid = 0.5 * (y(lambda_start_c_A) + y(lambda_stop_c_A)) * (lambda_stop_c_A - lambda_start_c_A)

    # Calculate area in between the pseudocontinuum and the spectrum!! 
    A_line = A_trapezoid - A_spec
    
    # Compute the EW, taking the level of the pseudocontinuum in the centre of the passband to be the continuum level
    EW_A = A_line / f_mean_c
    
    # ###################################################
    # # Plot
    # ###################################################
    # fig, ax = plt.subplots(figsize=(15, 5))
    # ax.plot(lambda_vals_rest_A, spec, color="grey")
    # ax.axvline(lambda_start_b_A, color="blue", label="Blue passband")
    # ax.axvline(lambda_stop_b_A, color="blue")
    # ax.axvline(lambda_start_r_A, color="red", label="Red passband")
    # ax.axvline(lambda_stop_r_A, color="red")
    # ax.axvline(lambda_start_c_A, color="grey", label="Central passband")
    # ax.axvline(lambda_stop_c_A, color="grey")
    # ax.plot([lambda_start_b_A, lambda_stop_b_A], [f_mean_b, f_mean_b], linewidth=3, color="black", label="Mean flux in blue/red passband")
    # ax.plot([lambda_start_r_A, lambda_stop_r_A], [f_mean_r, f_mean_r], linewidth=3, color="black")
    # ax.plot([lambda_mid_b_A, lambda_mid_r_A], [f_mean_b, f_mean_r], ls=":", color="black", label="Pseudocontinuum")
    # ax.fill_between([lambda_mid_c_A - EW / 2, lambda_mid_c_A + EW / 2], [y(lambda_mid_c_A - EW / 2), y(lambda_mid_c_A + EW / 2)], color="green", alpha=0.3)
    # ax.set_xlim([3900, 4200])
    # ax.set_ylim([0, None])
    # ax.set_xlabel(r"$\lambda$ (rest)")
    # ax.legend()
    
    return EW_A


In [59]:
###################################################
# Function for computing the D4000Å break strength
# as per Balogh (1999)
###################################################
def compute_D4000(spec, lambda_vals_A, z):

    # De-redshift the input spectrum
    lambda_vals_rest_A = lambda_vals_A / (1 + z) 

    # Compute the D4000Å break
    # Definition from Balogh+1999 (see here: https://arxiv.org/pdf/1611.07050.pdf, page 3)
    lambda_start_b_A = 3850
    lambda_stop_b_A = 3950
    lambda_start_r_A = 4000
    lambda_stop_r_A = 4100
    lambda_start_b_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_start_b_A))
    lambda_stop_b_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_stop_b_A))
    lambda_start_r_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_start_r_A))
    lambda_stop_r_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_stop_r_A))
    N_b = lambda_stop_b_idx - lambda_start_b_idx
    N_r = lambda_stop_r_idx - lambda_start_r_idx

    # Convert datacube & variance cubes to units of F_nu
    F_lambda = np.copy(spec)
    F_nu = F_lambda * lambda_vals_rest_A**2 / (constants.c * 1e10)

    num = np.nanmean(F_nu[lambda_start_r_idx:lambda_stop_r_idx], axis=0)
    denom = np.nanmean(F_nu[lambda_start_b_idx:lambda_stop_b_idx], axis=0)
    err_denom = 1 / N_b * np.sqrt(np.nansum(F_nu_err[lambda_start_b_idx:lambda_stop_b_idx]**2, axis=0))

    d4000 = num / denom

    # ###################################################
    # # Plot
    # ###################################################
    # fig, ax = plt.subplots(figsize=(15, 5))
    # ax.plot(lambda_vals_rest_A, spec, color="grey")
    # ax.axvline(lambda_start_b_A, color="blue", label="Blue passband")
    # ax.axvline(lambda_stop_b_A, color="blue")
    # ax.axvline(lambda_start_r_A, color="red", label="Red passband")
    # ax.axvline(lambda_stop_r_A, color="red")
    # ax.set_ylim([0, None])
    # ax.set_xlabel(r"$\lambda$ (rest)")
    # ax.legend()
    
    return d4000

In [ ]:
###################################################
# Settings
###################################################
bin_edges, bin_widths = get_bin_edges_and_widths(isochrones="Padova")
_, _, metallicities, ages = load_ssp_templates(isochrones="Padova")

In [65]:
###################################################
# Compute both quantities for bursty SFHs
###################################################
z = 0
fig, ax = plt.subplots()

for mm in range(3):
    hdelta_vals = []
    d4000_vals = []
    for aa in range(74):
        sfh_mw = np.zeros((3, 74))
        sfh_mw[mm, aa] = 1e10

        spec, spec_err, lambda_vals_A = create_mock_spectrum(
            sfh_mass_weighted=sfh_mw,
            agn_continuum=False,
            isochrones="Padova", z=z, SNR=1e5, sigma_star_kms=200,
            plotit=False)

        hdelta_vals.append(compute_Hdelta(spec, lambda_vals_A, z))
        d4000_vals.append(compute_D4000(spec, lambda_vals_A, z))

    ax.scatter(d4000_vals, hdelta_vals, label=f"metal idx = {mm}")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
###################################################
# Compute both quantities for constant SFHs
###################################################
z = 0
fig, ax = plt.subplots()


for mm in range(3):
    hdelta_vals = []
    d4000_vals = []
    for aa in range(1, 74):
        sfr = np.zeros((3, 74))
        sfr[mm, :aa] = 1
        sfh_mw = sfr * bin_widths[None, :]

        spec, spec_err, lambda_vals_A = create_mock_spectrum(
            sfh_mass_weighted=sfh_mw,
            agn_continuum=False,
            isochrones="Padova", z=z, SNR=1e5, sigma_star_kms=200,
            plotit=False)

        hdelta_vals.append(compute_Hdelta(spec, lambda_vals_A, z))
        d4000_vals.append(compute_D4000(spec, lambda_vals_A, z))

    ax.scatter(d4000_vals, hdelta_vals, label=f"metal idx = {mm}")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
###################################################
# Compute both quantities for "continuous" SFHs
# i.e. exponentially decaying SFHs
###################################################
# SFR \propto exp(-gamma t) where gamma ranges from 0 - 1 and t is in Gyr
for gamma in np.linspace(0, 1, 4):
    for t in ages / 1e9:
        sfr = np.zeros(N_metallicities, N_ages)
        ...
        
